In [ ]:
import os
os.chdir('/home')

import numpy as np
import pandas as pd
import gym

from kerasgym import preprocessing
from kerasgym import models
from kerasgym import exploration
from kerasgym import agents

from keras import optimizers

In [ ]:
# CARTPOLE

env = gym.make('CartPole-v1')
env.reset()

base_config = {
    'in_shape': env.observation_space.shape,
    'layer_sizes': [64, 32, 16],
    'activation': 'relu'
}
base_model = models.base.dense_model_base(**base_config)

dqn_config = {
    'gamma': 0.99,
    'tau': 1.0,
    'optimizer': optimizers.Adam(lr=0.001)
}
model = models.DQNModel(base_model, **dqn_config)

schedule = exploration.schedules.LinearDecay(1.0, 0.1, 500, -1)
actor = exploration.actors.EpsilonGreedyActor(epsilon_schedule=schedule)
buffer_size = 10000
batch_size = 32

agent = agents.Agent(env, state_pipeline=[], model=model, actor=actor, 
                     buffer_size=buffer_size, batch_size=batch_size,
                     report_freq=10, warmup_length=0)
agent.reset()
agent.run_indefinitely()